In [1]:
% matplotlib inline
from __future__ import (division, 
                        print_function)

import os
import sys
import copy
import fnmatch
import warnings
import collections

import numpy as np
import scipy
try:
    from scipy.stats import scoreatpercentile
except:
    scoreatpercentile = False
from scipy.interpolate import interp1d
import cPickle as pickle

# Astropy
from astropy.io import fits
from astropy    import units as u
from astropy.stats import sigma_clip
from astropy.table import Table, Column
from astropy.utils.console import ProgressBar

# Read in the FITS tables for the median profiles

In [5]:
ls profiles/*.fits

profiles/dr1_nonbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_mbin0.fits*
profiles/dr1_nonbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_mbin1.fits*
profiles/dr1_nonbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_mbin2.fits*
profiles/dr1_nonbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_minnA.fits*
profiles/dr1_nonbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_minnB.fits*
profiles/dr1_nonbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_minnC.fits*
profiles/dr1_nonbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_mtotA.fits*
profiles/dr1_nonbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_mtotB.fits*
profiles/dr1_nonbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_mtotC.fits*
profiles/dr1_nonbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_zbin1.fits*
profiles/dr1_nonbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_zbin2.fits*
profiles/dr1_redbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_mbin0.fits*
profiles/dr1_redbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_mbin1.fits*
profiles/dr1_redbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_mbin2.fits*
profiles/dr1_redbcg_isedfit_mass_f

In [6]:
dirProf = 'profiles'

rMbin0 = Table.read(os.path.join(dirProf, 'dr1_redbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_mbin1.fits'))
rMbin1 = Table.read(os.path.join(dirProf, 'dr1_redbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_mbin2.fits'))
rMbin2 = Table.read(os.path.join(dirProf, 'dr1_redbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_mbin3.fits'))

rM100C = Table.read(os.path.join(dirProf, 'dr1_redbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_mtotC.fits'))
rM10C = Table.read(os.path.join(dirProf, 'dr1_redbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_minnC.fits'))

nMbin0 = Table.read(os.path.join(dirProf, 'dr1_nonbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_mbin0.fits'))
nMbin1 = Table.read(os.path.join(dirProf, 'dr1_nonbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_mbin1.fits'))
nMbin2 = Table.read(os.path.join(dirProf, 'dr1_nonbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_mbin2.fits'))

nM100C = Table.read(os.path.join(dirProf, 'dr1_nonbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_mtotC.fits'))
nM10C = Table.read(os.path.join(dirProf, 'dr1_nonbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_minnC.fits'))

hMbin0 = Table.read(os.path.join(dirProf, 'hscMassive_mbin0.fits'))
hMbin1 = Table.read(os.path.join(dirProf, 'hscMassive_mbin1.fits'))
hMbin2 = Table.read(os.path.join(dirProf, 'hscMassive_mbin2.fits'))

In [7]:
rCol = rMbin0['rkpc'] ** 4.0

In [9]:
hMbin0.colnames

['rkpc',
 'medProfLow',
 'medProfUpp',
 'medProf',
 'avgProfLow',
 'avgProfUpp',
 'avgProf',
 'stdProf']

# Formatting the output string

In [10]:
format1 = "%4.1f & $%5.2f\substack{+%4.2f \\\\ -%4.2f}$ &"
format2 = "$%5.2f\substack{+%4.2f \\\\ -%4.2f}$ &"
format3 = "$%5.2f\substack{+%4.2f \\\\ -%4.2f}$ \\\\"

In [12]:
lastM = 0.0
lastR = 0.0

for ii, rr in enumerate(rCol):
    # redbcg Mbin0
    col2 = hMbin0[ii]['avgProf']
    col2p = (hMbin0[ii]['avgProfUpp'] - hMbin0[ii]['avgProf'])
    col2m = (hMbin0[ii]['avgProf'] - hMbin0[ii]['avgProfLow'])
    # redbcg Mbin1
    col3 = hMbin1[ii]['avgProf']
    col3p = (hMbin1[ii]['avgProfUpp'] - hMbin1[ii]['avgProf'])
    col3m = (hMbin1[ii]['avgProf'] - hMbin1[ii]['avgProfLow'])
    # redbcg Mbin2
    col4 = hMbin2[ii]['avgProf']
    col4p = (hMbin2[ii]['avgProfUpp'] - hMbin2[ii]['avgProf'])
    col4m = (hMbin2[ii]['avgProf'] - hMbin2[ii]['avgProfLow'])
    
    # redbcgM100 matched 
    col5 = rM100C[ii]['avgProf']
    col5p = (rM100C[ii]['avgProfUpp'] - rM100C[ii]['avgProf'])
    col5m = (rM100C[ii]['avgProf'] - rM100C[ii]['avgProfLow'])
    # nonbcgM100 matched 
    col6 = nM100C[ii]['avgProf']
    col6p = (nM100C[ii]['avgProfUpp'] - nM100C[ii]['avgProf'])
    col6m = (nM100C[ii]['avgProf'] - nM100C[ii]['avgProfLow'])
    
    # redbcgM10 matched
    col7 = rM10C[ii]['avgProf']
    col7p = (rM10C[ii]['avgProfUpp'] - rM10C[ii]['avgProf'])
    col7m = (rM10C[ii]['avgProf'] - rM10C[ii]['avgProfLow'])
    # redbcgM10 matched
    col8 = nM10C[ii]['avgProf']
    col8p = (nM10C[ii]['avgProfUpp'] - nM10C[ii]['avgProf'])
    col8m = (nM10C[ii]['avgProf'] - nM10C[ii]['avgProfLow'])
    
    if (ii == 0) or ((rr <= 5.0) and 
                     ((rr - lastR) >= 0.3) and 
                     (np.abs(col5 - lastM) > 0.03)) or ((rr <= 20.0) and 
                     ((rr - lastR) >= 1.3) and 
                     (np.abs(col5 - lastM) > 0.05)) or ((rr < 150.0) and 
                     ((rr - lastR) >= 3.5) and 
                     (np.abs(col5 - lastM) > 0.06)):
        string1 = format1 % (rr, col2, col2p, col2m)
        string2 = format2 %(col3, col3p, col3m)
        string3 = format2 %(col4, col4p, col4m)
        string4 = format2 %(col5, col5p, col5m)
        string5 = format2 %(col6, col6p, col6m)
        string6 = format2 %(col7, col7p, col7m)
        string7 = format3 %(col8, col7p, col7m)
        print(string1 + string2 + string3 + string4 + string5 + string6 + string7)
                     
        lastM = col5
        lastR = rr


 0.0 & $ 9.23\substack{+0.00 \\ -0.00}$ &$ 9.31\substack{+0.00 \\ -0.01}$ &$ 9.32\substack{+0.01 \\ -0.01}$ &$ 9.31\substack{+0.02 \\ -0.02}$ &$ 9.34\substack{+0.01 \\ -0.01}$ &$ 9.31\substack{+0.02 \\ -0.02}$ &$ 9.34\substack{+0.02 \\ -0.02}$ \\
 0.6 & $ 9.20\substack{+0.00 \\ -0.00}$ &$ 9.28\substack{+0.00 \\ -0.01}$ &$ 9.29\substack{+0.01 \\ -0.01}$ &$ 9.27\substack{+0.02 \\ -0.02}$ &$ 9.31\substack{+0.01 \\ -0.01}$ &$ 9.28\substack{+0.02 \\ -0.02}$ &$ 9.31\substack{+0.02 \\ -0.02}$ \\
 1.0 & $ 9.16\substack{+0.00 \\ -0.00}$ &$ 9.24\substack{+0.00 \\ -0.00}$ &$ 9.26\substack{+0.01 \\ -0.01}$ &$ 9.24\substack{+0.02 \\ -0.02}$ &$ 9.27\substack{+0.01 \\ -0.01}$ &$ 9.25\substack{+0.02 \\ -0.02}$ &$ 9.27\substack{+0.02 \\ -0.02}$ \\
 1.4 & $ 9.12\substack{+0.00 \\ -0.00}$ &$ 9.20\substack{+0.00 \\ -0.00}$ &$ 9.23\substack{+0.01 \\ -0.01}$ &$ 9.20\substack{+0.02 \\ -0.02}$ &$ 9.23\substack{+0.01 \\ -0.01}$ &$ 9.21\substack{+0.02 \\ -0.01}$ &$ 9.23\substack{+0.02 \\ -0.01}$ \\
 1.7 & $ 9.0